# Day 5 - Lab 1: Foundations of AI Agents - Tool-Using Agents (Solution)

**Objective:** Introduce the fundamental concepts of AI agents by building agents that can use external tools to accomplish tasks they cannot perform on their own.

**Introduction:**
This solution notebook provides the complete code and explanations for building your first agents. It covers the OpenAI Assistants API for a high-level introduction and then dives into the LangChain framework for more control and flexibility, culminating in a multi-tool agent.

## Step 1: Setup

**Explanation:**
We ensure all necessary libraries for this lab are installed. `langchain` and its related packages provide the core framework for building agents, while `tavily-python` is the SDK for the Tavily Search API, which our agent will use as a tool.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
# This helper will install packages if they are not found
import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        %pip install -q {package}

install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('tavily-python')

from utils import setup_llm_client
# We will use the OpenAI provider for this lab
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Using the OpenAI Assistants API

**Explanation:**
The Assistants API is OpenAI's high-level framework for building stateful agents. 
1.  **`client.beta.assistants.create`**: We define the agent's persona and capabilities. We give it instructions and enable the `code_interpreter` tool, which allows it to execute Python code in a sandboxed environment to perform tasks like calculations.
2.  **`client.beta.threads.create`**: Assistants are stateful. A `thread` represents a single conversation, allowing the assistant to remember previous messages.
3.  **`client.beta.threads.messages.create`**: We add our user's question to the conversation thread.
4.  **`client.beta.threads.runs.create`**: A `run` is the process of the assistant executing to respond to the messages in the thread. It's an asynchronous operation, so we poll its status until it's `completed`.
5.  **`client.beta.threads.messages.list`**: Once the run is complete, we retrieve the full list of messages from the thread, which now includes the assistant's response.

In [ ]:
import time
from openai import OpenAI

if api_provider != 'openai':
    print("This challenge requires an OpenAI client. Please set up your client accordingly.")
else:
    # 1. Create an assistant
    assistant = client.beta.assistants.create(
        name="Math Tutor",
        instructions="You are a personal math tutor. Write and run code to answer math questions.",
        tools=[{"type": "code_interpreter"}],
        model=model_name
    )
    print(f"Assistant created with ID: {assistant.id}")

    # 2. Create a thread
    thread = client.beta.threads.create()
    print(f"Thread created with ID: {thread.id}")

    # 3. Add a message to the thread
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="What is `(123 * 4) + (567 / 8)`?"
    )

    # 4. Create a run and wait for completion
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    
    while run.status != 'completed':
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        print(f"Run status: {run.status}")

    # 5. Retrieve and print the messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in reversed(messages.data): # Reverse to show in chronological order
        print(f"\n{msg.role.capitalize()}: {msg.content[0].text.value}")

### Challenge 2 (Intermediate): Building a LangChain Agent with One Tool

**Explanation:**
LangChain provides a more modular and flexible way to build agents. 
1.  **Tool:** We instantiate `TavilySearchResults`, which is a pre-built LangChain component that knows how to call the Tavily API.
2.  **Prompt:** The prompt is crucial. `ChatPromptTemplate.from_messages` creates a template for the conversation. The key part is the `("placeholder", "{agent_scratchpad}")`. The `agent_scratchpad` is a special variable where the agent's internal monologue (its thoughts, tool calls, and tool outputs) is stored. This allows the agent to reason about its actions.
3.  **Agent:** `create_tool_calling_agent` binds the LLM, the tools, and the prompt together into a runnable agent.
4.  **AgentExecutor:** This is the runtime for the agent. It takes the agent and the tools and handles the logic of calling the agent, parsing its output to see if it wants to use a tool, executing the tool, and passing the result back to the agent to continue its reasoning.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# We need to use a LangChain LLM wrapper
llm = ChatOpenAI(model=model_name)

# 1. Instantiate the Tavily search tool
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

# 2. Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 3. Create the agent
agent = create_tool_calling_agent(llm, tools, prompt)

# 4. Create the AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 5. Invoke the agent with a question
question = "What was the score of the last Super Bowl?"
result = agent_executor.invoke({"input": question})

print(f"\nFinal Answer: {result['output']}")

### Challenge 3 (Advanced): Building a Multi-Tool Agent

**Explanation:**
This challenge demonstrates the core reasoning power of an agent. 
1.  **`@tool` Decorator:** LangChain provides a simple `@tool` decorator to turn any Python function into a tool that an agent can use. The function's docstring is very important, as the agent uses it to understand what the tool does.
2.  **Tool List:** We create a new list containing both our custom `multiply` tool and the pre-built `TavilySearchResults` tool.
3.  **Reasoning:** When we create the new `AgentExecutor` with this list, the agent now has a choice. When invoked, the LLM will look at the user's question and the docstrings of all available tools. It then decides which tool, if any, is the most appropriate for the task. For "25 * 48", it will see the "Multiplies two integers" docstring and choose the `multiply` tool. For "Who is the CEO of Apple?", it will recognize that it needs external information and choose the search tool.

In [ ]:
from langchain_core.tools import tool

# 1. Define your custom calculator tool
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two integers together. Use this for math questions involving multiplication."""
    return a * b

# 2. Create the new list of tools
multi_tool_list = [TavilySearchResults(max_results=2), multiply]

# 3. Create the new multi-tool agent and executor
# The prompt is the same, the agent just has more tools to choose from.
multi_tool_agent = create_tool_calling_agent(llm, multi_tool_list, prompt)
multi_tool_executor = AgentExecutor(agent=multi_tool_agent, tools=multi_tool_list, verbose=True)

# 4. Invoke the agent with a math question
math_question = "What is 25 * 48?"
math_result = multi_tool_executor.invoke({"input": math_question})
print(f"\nQuery: {math_question}\nFinal Answer: {math_result['output']}\n")

# 5. Invoke the agent with a search question
search_question = "Who is the current CEO of Apple?"
search_result = multi_tool_executor.invoke({"input": search_question})
print(f"\nQuery: {search_question}\nFinal Answer: {search_result['output']}")

## Lab Conclusion

Congratulations! You have successfully built your first AI agents. You started with the high-level OpenAI Assistants API and then moved to the more flexible LangChain framework. You've learned how to give agents tools to extend their capabilities and, most importantly, how to build an agent that can reason about which tool to use for a specific task. This is the foundational skill for all advanced agentic workflows we will explore in the coming days.